In [42]:
# Implementation of a Sum-HMM for a ML assignement - December 2017/January 2018

import numpy as np
import matplotlib.pyplot as plt

In [43]:
# Number of players
n = 1

# Number of games for each player
K = 10

# Probability of observing an outcome
p = 0.5

# Decides which distribution is used for the dice: 
# 0 and 1 for test distributions, 2 for uniform, 3 for random
dist_option = 3

dice_dist = np.zeros((2*K,6))

# Generates distributions for which an outcome of 1 is certain for each die
if (dist_option == 0):
    
    dice_dist[:,0] = 1

# Generates test distributions
if (dist_option == 1):
    
    for i in range(K):
        dice_dist[2*i][0] = 1
        dice_dist[2*i+1][1] = 1
    
# Generates uniform distributions for the dice
if (dist_option == 2):
    
    dice_dist = 0.16666666667*np.ones((2*K,6))

# Generates random distributions for all the dice
if (dist_option == 3):
    
    key = np.arange(6)

    for i in range(2*K):
        np.random.shuffle(key)
        sum_proba = 0
        for j in range(5):
            random_proba = np.random.uniform(0,1-sum_proba)
            dice_dist[i][key[j]] = random_proba
            sum_proba += random_proba
        dice_dist[i][key[5]] = 1 - sum_proba
        


In [44]:
# Stores the sequences
X = np.zeros((n,K))

# Stores the sums
S = np.zeros(n)

# Main loop: the n players each play K games
for i in range(n):
    
    # Starting table: can be 0 or 1 with equal probability
    table_number = np.random.randint(0,2)
    #table_number = 0
    
    for j in range(K):
        
        # Computes the outcome of the die at this table
        random_number = np.random.uniform(0,1)
        sum_proba = 0
        for k in range(6):
            sum_proba += dice_dist[2*j+table_number][k]
            if (random_number < sum_proba):
                outcome = k + 1
                break
        
        # Checks if the outcome is observed
        S[i] += outcome 
        if (random_number < p):
            X[i][j] = outcome
        else:
            X[i][j] = 0
            
        # Jumps to the next table
        if (random_number >= 0.25):
            table_number = 1 - table_number

X = X.astype(int)
S = S.astype(int)
print(X)
print(S)           

[[2 0 0 3 3 0 2 5 0 2]]
[34]


In [45]:
# Computes P(Zk) and P(Xk) as K*2 and K*6 arrays
PZ = np.zeros((K,2))
PX = np.zeros((K,6))

# Initial probability for the first table
PZ[0][0] = 0.5
PZ[0][1] = 0.5

PX[0][0] = 0.5*dice_dist[0][0] + 0.5*dice_dist[1][0]
PX[0][1] = 0.5*dice_dist[0][1] + 0.5*dice_dist[1][1]
PX[0][2] = 0.5*dice_dist[0][2] + 0.5*dice_dist[1][2]
PX[0][3] = 0.5*dice_dist[0][3] + 0.5*dice_dist[1][3]
PX[0][4] = 0.5*dice_dist[0][4] + 0.5*dice_dist[1][4]
PX[0][5] = 0.5*dice_dist[0][5] + 0.5*dice_dist[1][5]

for i in range(K-1):
    
    PZ[i+1][0] = 0.25*PZ[i][0] + 0.75*PZ[i][1]
    PZ[i+1][1] = 0.75*PZ[i][0] + 0.25*PZ[i][1]
    
    PX[i+1][0] = PZ[i+1][0]*dice_dist[2*(i+1)][0] + PZ[i+1][1]*dice_dist[2*(i+1)+1][0]
    PX[i+1][1] = PZ[i+1][0]*dice_dist[2*(i+1)][1] + PZ[i+1][1]*dice_dist[2*(i+1)+1][1]
    PX[i+1][2] = PZ[i+1][0]*dice_dist[2*(i+1)][2] + PZ[i+1][1]*dice_dist[2*(i+1)+1][2]
    PX[i+1][3] = PZ[i+1][0]*dice_dist[2*(i+1)][3] + PZ[i+1][1]*dice_dist[2*(i+1)+1][3]
    PX[i+1][4] = PZ[i+1][0]*dice_dist[2*(i+1)][4] + PZ[i+1][1]*dice_dist[2*(i+1)+1][4]
    PX[i+1][5] = PZ[i+1][0]*dice_dist[2*(i+1)][5] + PZ[i+1][1]*dice_dist[2*(i+1)+1][5]

In [46]:
# Computes P(S) (for 2.2)

# Stores in memory the partial sums for which we already have computed the value
partial_sums = -np.ones((6*K,K-1))

# Recursive function to compute P(S) for a given S
def rec(S,k):
    
    result = 0
    
    for i in range(6):
        
        if (partial_sums[S-i-2][k-2] != -1):
            result += partial_sums[S-i-2][k-2]*PX[k-1][i]
            
        # Checks if the partial sum is within the acceptable boundaries
        elif (((S-i-1) >= (k-1)) & ((S-i-1) <= 6*(k-1))):
            
            if (k == 2):
                result += PX[1][i]*PX[0][S-i-2]
                #partial_sums[i][0] = PX[1][i]*PX[0][S-i-2]
                
            else:
                result += PX[k-1][i]*rec(S-i-1,k-1)
                partial_sums[S-i-2][k-2] = rec(S-i-1,k-1)
                
    return result

# Stores the probabilities P(S) computed. This array sums to 1.
PS = np.zeros(5*K+1)

# Computes P(S) for all possible values of S
def PPS():
    
    for S in range(K,6*K+1):
        PS[S-K] = rec(S,K)
        
#PPS()
#print(PS)
#print(sum(PS))

In [47]:
# Computes P(S,X|Theta) (for 2.4)

# Stores in memory the partial sums for which we already have computed the value
#partial_sums_obs = -np.ones((6*K,K-1))

# Recursive function to compute P(S,X\Theta) for a given S,X
def rec_obs(S,X,k):
    
    result = 0
    
    # Checks if the outcome has been observed for this table
    if (X[k-1] != 0):
        
        # Checks if this partial sum has already been computed before
        #if (partial_sums_obs[S-X[k-1]-1][k-2] != -1):
            
            #result = p*PX[k-1][X[k-1]-1]*partial_sums_obs[S-X[k-1]-1][k-2]
            
        # Checks if the partial sum is within the acceptable boundaries
        if (((S-X[k-1]) >= (k-1)) & ((S-X[k-1]) <= 6*(k-1))):
            
            if (k == 2):
            
                # Checks if the first outcome is observed
                if (X[0] == 0):
                    
                    result = p*(1-p)*PX[0][S-X[k-1]-1]*PX[1][X[k-1]-1]
                    
                elif (X[0] == S-X[k-1]):
                    
                    result = p*p*PX[0][S-X[k-1]-1]*PX[1][X[k-1]-1]
            
            else:
            
                result = p*PX[k-1][X[k-1]-1]*rec_obs(S-X[k-1],X,k-1)
                #partial_sums_obs[S-X[k-1]-1][k-2] = rec_obs(S-X[k-1],X,k-1)
        
    else:
        
        for i in range(6):
        
            # Checks if this partial sum has already been computed before
            #if (partial_sums_obs[S-i-2][k-2] != -1):
                
                #result += partial_sums_obs[S-i-2][k-2]*PX[k-1][i]
            
            # Checks if the partial sum is within the acceptable boundaries
            if (((S-i-1) >= (k-1)) & ((S-i-1) <= 6*(k-1))):
            
                if (k == 2):
                    
                    # Checks if the first outcome is observed
                    if (X[0] == 0):
                        
                        result += (1-p)*(1-p)*PX[1][i]*PX[0][S-i-2]
                        
                    elif (X[0] == S-i-1):
                    
                        result += (1-p)*p*PX[1][i]*PX[0][S-i-2]
                
                else:
                    
                    result += (1-p)*PX[k-1][i]*rec_obs(S-i-1,X,k-1)
                    #partial_sums_obs[S-i-2][k-2] = rec_obs(S-i-1,X,k-1)
                
    return result


# Index of the player for which we want to have this probability
pl = 0

PSX = rec_obs(S[pl],X[pl],K)

print(PSX)

"""
# FIRST TEST - CHECKS WHICH SUMS ARE NOT REACHABLE DEPENDING ON X

# Stores the probabilities P(S) computed
PS_obs = np.zeros(5*K+1)

# Computes P(S) for all possible values of S
def PPS_obs():
    
    for S in range(K,6*K+1):
        PS_obs[S-K] = rec_obs(S,X[pl],K)
        
print(X[pl])
PPS_obs()
print(PS_obs)
print(sum(PS_obs))
"""

"""
# SECOND TEST - CHECKS THAT PROBABILITIES SUM TO 1

# Stores the probabilities P(S,X) computed for all possible values of S and X
PS_obs_full = np.zeros((5*K+1,pow(7,K)))

# Computes P(S) for all possible values of S and X, only for the case K=5
def PPS_obs_full():
    
    # Case where K=5
    for S in range(K,6*K+1):
        counter = 0
        for a in range(7):
            for b in range(7):
                for c in range(7):
                    for d in range(7):
                        for e in range(7):
                            X = np.array([a,b,c,d,e])
                            PS_obs_full[S-K,counter] = rec_obs(S,X,K)
                            counter += 1
            
        
PPS_obs_full()
print("sum of the probabilities for K=5: " + str(sum(sum(PS_obs_full))))
"""


2.45313084443e-09


'\n# SECOND TEST - CHECKS THAT PROBABILITIES SUM TO 1\n\n# Stores the probabilities P(S,X) computed for all possible values of S and X\nPS_obs_full = np.zeros((5*K+1,pow(7,K)))\n\n# Computes P(S) for all possible values of S and X, only for the case K=5\ndef PPS_obs_full():\n    \n    # Case where K=5\n    for S in range(K,6*K+1):\n        counter = 0\n        for a in range(7):\n            for b in range(7):\n                for c in range(7):\n                    for d in range(7):\n                        for e in range(7):\n                            X = np.array([a,b,c,d,e])\n                            PS_obs_full[S-K,counter] = rec_obs(S,X,K)\n                            counter += 1\n            \n        \nPPS_obs_full()\nprint("sum of the probabilities for K=5: " + str(sum(sum(PS_obs_full))))\n'

In [48]:
# Samples Zi

# Computes P(S,X|Z)
def rec_obs_Z(S,X,Z,k):
    
    result = 0
    
    # Checks if value of Zk is known. If yes continues as for P(S,X)
    if (Z[k-1] == 0):
    
        # Checks if the outcome has been observed for this table
        if (X[k-1] != 0):
            
            # Checks if the partial sum is within the acceptable boundaries
            if (((S-X[k-1]) >= (k-1)) & ((S-X[k-1]) <= 6*(k-1))):
            
                if (k == 2):
            
                    # Checks if the first outcome is observed
                    if (X[0] == 0):
                    
                        result = p*(1-p)*PX[0][S-X[k-1]-1]*PX[1][X[k-1]-1]
                    
                    elif (X[0] == S-X[k-1]):
                    
                        result = p*p*PX[0][S-X[k-1]-1]*PX[1][X[k-1]-1]
            
                else:
            
                    result = p*PX[k-1][X[k-1]-1]*rec_obs_Z(S-X[k-1],X,Z,k-1)
        
        else:
        
            for i in range(6):
            
                # Checks if the partial sum is within the acceptable boundaries
                if (((S-i-1) >= (k-1)) & ((S-i-1) <= 6*(k-1))):
            
                    if (k == 2):
                    
                        # Checks if the first outcome is observed
                        if (X[0] == 0):
                        
                            result += (1-p)*(1-p)*PX[1][i]*PX[0][S-i-2]
                        
                        elif (X[0] == S-i-1):
                    
                            result += (1-p)*p*PX[1][i]*PX[0][S-i-2]
                
                    else:
                    
                        result += (1-p)*PX[k-1][i]*rec_obs_Z(S-i-1,X,Z,k-1)
    
    # In this case, the value of Zk is known.
    else:
        
        # Checks if the outcome has been observed for this table
        if (X[k-1] != 0):
            
            # Checks if the partial sum is within the acceptable boundaries
            if (((S-X[k-1]) >= (k-1)) & ((S-X[k-1]) <= 6*(k-1))):
            
                if (k == 2):
            
                    # Checks if the first outcome is observed
                    if (X[0] == 0):
                        
                        # Checks if Z1 is known
                        if (Z[0] == 0):
                    
                            result = p*(1-p)*PX[0][S-X[k-1]-1]*dice_dist[2*(k-1)+Z[k-1]-1,X[k-1]-1]
                        
                        else:
                            
                            result = p*(1-p)*dice_dist[Z[0]-1,S-X[k-1]-1]*dice_dist[2*(k-1)+Z[k-1]-1,X[k-1]-1]
                    
                    elif (X[0] == S-X[k-1]):
                        
                        # Checks if Z1 is known
                        if (Z[0] == 0):
                            
                            result = p*p*PX[0][S-X[k-1]-1]*dice_dist[2*(k-1)+Z[k-1]-1,X[k-1]-1]
                            
                        else:
                                
                            result = p*p*dice_dist[Z[0]-1,S-X[k-1]-1]*dice_dist[2*(k-1)+Z[k-1]-1,X[k-1]-1]
            
                else:
            
                    result = p*dice_dist[2*(k-1)+Z[k-1]-1,X[k-1]-1]*rec_obs_Z(S-X[k-1],X,Z,k-1)
        
        else:
        
            for i in range(6):
            
                # Checks if the partial sum is within the acceptable boundaries
                if (((S-i-1) >= (k-1)) & ((S-i-1) <= 6*(k-1))):
            
                    if (k == 2):
                    
                        # Checks if the first outcome is observed
                        if (X[0] == 0):
                        
                            # Checks if Z1 is known
                            if (Z[0] == 0):
                    
                                result = (1-p)*(1-p)*PX[0][S-i-2]*dice_dist[2*(k-1)+Z[k-1]-1,i]
                        
                            else:
                            
                                result = (1-p)*(1-p)*dice_dist[Z[0]-1,S-i-2]*dice_dist[2*(k-1)+Z[k-1]-1,i]
                    
                        elif (X[0] == S-X[k-1]):
                        
                            # Checks if Z1 is known
                            if (Z[0] == 0):
                            
                                result = p*(1-p)*PX[0][S-i-2]*dice_dist[2*(k-1)+Z[k-1]-1,i]
                            
                            else:
                                
                                result = p*(1-p)*dice_dist[Z[0]-1,S-i-2]*dice_dist[2*(k-1)+Z[k-1]-1,i]
                
                    else:
                    
                        result += (1-p)*dice_dist[2*(k-1)+Z[k-1]-1,i]*rec_obs_Z(S-i-1,X,Z,k-1)
    
    return result


"""
# THIRD TEST - CHECKS THAT P(ZK = 1 | S, X) + P(ZK = 2 | S, X) = 1

Z = np.array([0 for i in range(K)])
Z[K-1] = 1
pp_1 = (rec_obs_Z(S[pl],X[pl],Z,K)*PZ[K-1,0])/PSX
Z[K-1] = 2
pp_2 = (rec_obs_Z(S[pl],X[pl],Z,K)*PZ[K-1,1])/PSX
print(pp_1 + pp_2)
print("\n")
"""

Z = np.array([0 for i in range(K)])
p_1 = 0
p_2 = 0
prev_p_1 = 0
prev_p_2 = 0

# Initializes for ZK
Z[K-1] = 1
prev_p_1 = rec_obs_Z(S[pl],X[pl],Z,K)
p_1 = (prev_p_1*PZ[K-1,0])/PSX
Z[K-1] = 2
prev_p_2 = rec_obs_Z(S[pl],X[pl],Z,K)
p_2 = (prev_p_2*PZ[K-1,1])/PSX

print(p_1 + p_2)

# Samples Zi from S and X generated by our simulation
for i in range(K-1):
    
    if (Z[K-1-i] == 1):
        emission = 0.25
        prev_p = prev_p_1
    else:
        emission = 0.75
        prev_p = prev_p_2

    # Computes probability assuming that Zk = 1
    Z[K-2-i] = 1
    temp_1 = rec_obs_Z(S[pl],X[pl],Z,K)
    p_1 = (temp_1*PZ[K-2-i,0]*emission)/(prev_p*PZ[K-1-i,Z[K-1-i]-1])
    
    # Computes probability assuming that Zk = 2
    Z[K-2-i] = 2
    temp_2 = rec_obs_Z(S[pl],X[pl],Z,K)
    p_2 = (temp_2*PZ[K-2-i,1]*(1-emission))/(prev_p*PZ[K-1-i,Z[K-1-i]-1])
    
    prev_p_1 = temp_1
    prev_p_2 = temp_2
    
    # Samples a value for Zk
    rand_num = np.random.uniform(0,1)
    if rand_num < p_1:
        Z[K-2-i] = 1
    else:
        Z[K-2-i] = 2
    
    print(p_1 + p_2)

print(Z)


1.0
1.00884457397
0.77254107092
1.42406607602
0.990442348775
0.500003328415
0.690739093602
1.00746892007
0.00887038475752
1.27897686763
[1 2 1 2 1 1 2 1 1 2]
